<a href="https://colab.research.google.com/github/Samanga06/samanga/blob/main/emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import zipfile
from pathlib import Path
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [2]:
!pip install -q git+https://github.com/openai/whisper.git
import whisper

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 137.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 115.5 MB/s 

In [4]:
zip_path = "ravdass.zip"
audio_dir = "ravdass"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(audio_dir)

In [5]:
emotion_map = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}

mel_dir = Path("mel_spectrograms")
mel_dir.mkdir(exist_ok=True)

def save_spectrogram(wav_path, output_path):
    y, sr = librosa.load(wav_path, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    plt.figure(figsize=(2.24, 2.24), dpi=100)
    librosa.display.specshow(S_dB, sr=sr, cmap='magma')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

for wav_file in tqdm(Path(audio_dir).rglob("*.wav")):
    parts = wav_file.stem.split("-")
    if len(parts) < 3: continue
    emotion_id = parts[2]
    emotion = emotion_map.get(emotion_id)
    if emotion:
        out_folder = mel_dir / emotion
        out_folder.mkdir(parents=True, exist_ok=True)
        save_spectrogram(wav_file, out_folder / (wav_file.stem + ".png"))


2880it [05:18,  9.03it/s]


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_dataset = datasets.ImageFolder(str(mel_dir), transform=transform)
image_loader = DataLoader(image_dataset, batch_size=32, shuffle=True)

class EmotionCNN(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.base = models.resnet18(pretrained=False)
        self.base.fc = nn.Linear(self.base.fc.in_features, num_classes)

    def forward(self, x):
        return self.base(x)

In [3]:
transcripts = []
labels = []
model = whisper.load_model("tiny")

for wav_file in tqdm(Path(audio_dir).rglob("*.wav")):
    parts = wav_file.stem.split("-")
    emotion_id = parts[2]
    emotion = emotion_map.get(emotion_id)
    if emotion:
        result = model.transcribe(str(wav_file), fp16=False)
        transcripts.append(result['text'])
        labels.append(emotion)


NameError: name 'whisper' is not defined

In [1]:
!pip install torchtext
!pip install --upgrade torch==2.1.0 torchtext==0.16.0


In [2]:
import pandas as pd
text_df = pd.DataFrame({"text": transcripts, "label": labels})
text_df.to_csv("transcripts.csv", index=False)

# Step 5: RNN for Transcript Classification
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import GloVe
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab):
        self.texts = [torch.tensor([vocab.stoi.get(w, 0) for w in t.lower().split()]) for t in texts]
        self.labels = labels
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

def collate_fn(batch):
    texts, labels = zip(*batch)
    padded_texts = pad_sequence(texts, batch_first=True)
    return padded_texts, torch.tensor(labels)


NameError: name 'transcripts' is not defined

In [14]:
le = LabelEncoder()
label_tensor = torch.tensor(le.fit_transform(text_df['label']))


In [ ]:
vocab = GloVe(name='6B', dim=100)
text_data = TextDataset(text_df['text'].tolist(), label_tensor, vocab)
text_loader = DataLoader(text_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

class TextRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embed = nn.EmbeddingBag.from_pretrained(vocab.vectors)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embed(x)
        out, (hn, _) = self.rnn(embedded.unsqueeze(1))
        return self.fc(hn[-1])


In [ ]:
from docx import Document

doc = Document()
doc.add_heading("Emotion Recognition Results", 0)
doc.add_paragraph("Training and Validation Accuracy:")
doc.add_paragraph("CNN Accuracy: 85%\nRNN Accuracy: 82%")  # Replace with real values

doc.save("emotion_report.docx")

In [ ]:
from google.colab import files
files.download("emotion_report.docx")